Download the speech and noise dataset from DEMAND and VCTK
- The file extraction is based on these two dataset if there is a change in dataset make sure the change the directory names
- This program can be used to generate both training and testing datasets for both the GAN and CNN models.

https://www.kaggle.com/datasets/showmik50/vctk-dataset
You can extract the vctk dataset directly in the same folder as the create_dataset.ipynb

https://www.kaggle.com/datasets/chrisfilo/demand
Extract the demand dataset inside of a new folder called "noise"

In [5]:
import torch
import torchaudio
import torchaudio.functional as F
import math
import librosa
import wave
import os
import shutil
from scipy.io import wavfile
import math
import numpy as np
import scipy.signal as sps
from scipy.signal import butter, lfilter
import random
from pydub import AudioSegment, effects
import soundfile as sf
import openpyxl

import xlsxwriter

print(torch.__version__)
print(torchaudio.__version__)
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2.0.1+cpu
2.0.2+cpu


Step 1) Creating folders for mixtures at different SNR levels and including the original clean and noisy speech.

In [6]:
# Creating dataset folder and sub-folders
folder_name = "../Dataset/dataset" # change to change dataset name and directory if needed
path_clean = folder_name + "/clean/"
path_noisy = folder_name + "/noisy/"
path_0dB = folder_name + "/0dB/"
path_n3dB = folder_name + "/-3dB/"
path_n6dB = folder_name + "/-6dB/"
path_n9dB = folder_name + "/-9dB/"
path_3dB = folder_name + "/3dB/"
path_6dB = folder_name + "/6dB/"
path_9dB = folder_name + "/9dB/"
path_noise = folder_name + "/new_noise/" # Segement of the orignal noise
path3 = folder_name + "/all_info.xlsx"

In [7]:
isExist = os.path.exists(folder_name)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(folder_name)
   os.makedirs(folder_name + "/clean")
   os.makedirs(folder_name + "/noisy")
   os.makedirs(folder_name + "/0dB")
   os.makedirs(folder_name + "/-3dB")
   os.makedirs(folder_name + "/-6dB")
   os.makedirs(folder_name + "/-9dB")
   os.makedirs(folder_name + "/3dB")
   os.makedirs(folder_name + "/6dB")
   os.makedirs(folder_name + "/9dB")
   print("the %s directory is created!", folder_name)

isExist2 = os.path.exists(path_noise)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(path_noise)
   os.makedirs(path_noise + "/0dB")
   os.makedirs(path_noise + "/-3dB")
   os.makedirs(path_noise + "/-6dB")
   os.makedirs(path_noise + "/-9dB")
   os.makedirs(path_noise + "/3dB")
   os.makedirs(path_noise + "/6dB")
   os.makedirs(path_noise + "/9dB")
   print("the %s directory is created!", folder_name)

isExist3 = os.path.exists(path3)
if not isExist3:
   workbook1 = xlsxwriter.Workbook(path3)
   worksheet1 = workbook1.add_worksheet()
   worksheet1.write(0, 0, "Sample Number")
   worksheet1.write(0, 1, "Name")
   worksheet1.write(0, 2, "Length(s)")
   worksheet1.write(0, 3, "Type of Speech")
   worksheet1.write(0, 4, "SNR(dB)")
   worksheet1.write(0, 5, "Noise Type")
   worksheet1.write(0, 6, "Noise Channel")
   worksheet1.write(0, 7, "Speaker ID")
   worksheet1.write(0, 8, "Passage ID")
   worksheet1.write(0, 9, "Noise Start Time(s)")
   print("The excel file 1 is created!")
   workbook1.close()

the %s directory is created! ../Dataset/predict_dataset
the %s directory is created! ../Dataset/predict_dataset
The excel file 1 is created!


Step 2) Locations of speech and noise

In [8]:
speech_fpath = "VCTK-Corpus/VCTK-Corpus/wav48/" # Change this for different speech dataset
speech_file_entries = os.listdir(speech_fpath)
print("Amount of speech folders: " + str(len(speech_file_entries)))

Amount of speech folders: 109


In [9]:
noise_fpath = "noise/" # Change this for different noise dataset 

# List all items (files and directories) in the specified directory - Specific for the demand dataset to remove the 48k noise
items = os.listdir(noise_fpath)
# Iterate through the items and remove folders with "48K" in their names
for item in items:
    item_path = os.path.join(noise_fpath, item)
    
    if os.path.isdir(item_path) and "48k" in item.split("_")[1]:
        try:
            shutil.rmtree(item_path)  # Use os.rmdir to remove directories
            print(f"Removed: {item}")
        except Exception as e:
            print(f"Error removing {item}: {e}")
noise_file_entries = os.listdir(noise_fpath)
print("Amount of noise folders: " + str(len(noise_file_entries)))

Amount of noise folders: 17


Step 3) Normalise and downsample speech from 48K to 16K and normalise noise

In [10]:
def butter_lowpass(cutoff, fs, order=5): # Butterworth lowpass filter
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [11]:
def downsampling(clip): # Downsample the audio to 16k
    samplerate, data = wavfile.read(clip)
    Fs1 = samplerate
    Fs2 = 16000
    N = len(data)
    total_time = (N-1)/Fs1
    Max_Signal_Frequency =Fs2/2
    New_sample_amount = math.ceil(Fs2*total_time)
    Single_Channel = np.zeros(New_sample_amount)
    Bit_Check = wave.open(clip) # Check the bit depth of the audio
    bit_depth = Bit_Check.getsampwidth() * 8
    data = data/(2**(bit_depth-1))
    Original_signal = data
    Anti_Aliased_signal = np.array(butter_lowpass_filter(Original_signal,Max_Signal_Frequency,Fs1))
    Down_sampled_signal = np.array(sps.resample(Anti_Aliased_signal,New_sample_amount))
    Single_Channel = Down_sampled_signal
    Transformed_single_channel = Single_Channel.transpose()
    return Transformed_single_channel, Fs2

In [12]:
def normalise(inclip, outclip): # Normalise the audio
    rawsound = AudioSegment.from_wav(inclip)  
    normalizedsound = effects.normalize(rawsound)  
    normalizedsound.export(outclip, format = 'wav')

Step 4) Adding noise to speech

In [13]:
def adding_noise_to_speech(speech, noise, i):
    speech, _ = torchaudio.load(speech)
    noise, _ = torchaudio.load(noise)

    # From a random point in the noise waveform make the size of the noise the same as the speech
    first = random.randint(0, noise.shape[1] - speech.shape[1])
    noise = noise[:, first:first + speech.shape[1]]

    # Calculate the time range of the noise in minutes
    noise_start_time = first / 16000
    # At all SNR levels add the noise to the speech
    snr_dbs = torch.tensor([0, -3, -6, -9, 3, 6, 9], device=device)
    noisy_speeches = F.add_noise(speech.to(device), noise.to(device), snr_dbs)

    snr_db, noisy_speech = snr_dbs[0].item(), noisy_speeches[0:1]
    torchaudio.save(path_0dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[1].item(), noisy_speeches[1:2]
    torchaudio.save(path_n3dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[2].item(), noisy_speeches[2:3]
    torchaudio.save(path_n6dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[3].item(), noisy_speeches[3:4]
    torchaudio.save(path_n9dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[4].item(), noisy_speeches[4:5]
    torchaudio.save(path_3dB + "speech_" + str(i)+ ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[5].item(), noisy_speeches[5:6]
    torchaudio.save(path_6dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[6].item(), noisy_speeches[6:7]
    torchaudio.save(path_9dB + "speech_" + str(i) + ".wav", noisy_speech.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)

    # Save the noisy speech at different SNR levels
    for j, snr_db in enumerate(snr_dbs):
        snr_db = snr_db.item()
        noisy_speech = noisy_speeches[j:j+1]
        # Save the noise separately at each SNR level
        noise_at_snr = noisy_speech - speech.to(device)
        torchaudio.save(path_noise + "/" + str(snr_db) + "dB/speech_" + str(i) + ".wav", noise_at_snr.cpu(), 16000, encoding="PCM_S", bits_per_sample=16)
        
    return noise_start_time


Step 5) Create all and classification data with excel information

In [14]:
n = 1500 # Change the number of samples created
for i in range (0, n):

    # Random number to select the clean speech file (speaker) used
    ran_1 = random.randrange(0, len(speech_file_entries) - 1)
    speech_file = speech_file_entries[ran_1]
    speech_enter = speech_fpath + speech_file + "/"
    speech_entries = os.listdir(speech_enter)

    # Random number to select the passage used
    ran_2 = random.randrange(0, len(speech_entries) - 1)
    select_speech = speech_enter + speech_entries[ran_2]

    # Selecting the noise
    ran_3 = random.randrange(0, len(noise_file_entries) - 1)
    noise_file = noise_file_entries[ran_3]
    noise_enter = noise_fpath + noise_file + "/" + noise_file.split("_")[0] + "/"
    noise_entries = os.listdir(noise_enter)

    # Random number to select the channel used
    ran_4 = random.randrange(0, len(noise_entries) - 1)
    select_noise = noise_enter + noise_entries[ran_4]

    # downsampling then normalising the clean speech
    data, samplerate = downsampling(select_speech)
    sf.write(path_clean + "speech_" + str(i + 1) + ".wav", data, samplerate, 'PCM_16')
    normalise(path_clean + "speech_" + str(i + 1) + ".wav", path_clean + "speech_" + str(i + 1) + ".wav")
    
    # normalising the noisy speech
    normalise(select_noise, path_noisy + "noisy_" + str(i + 1) + ".wav")

    # Combinding the clean speech and the noise
    noise_start_time = adding_noise_to_speech(path_clean + "speech_" + str(i + 1) + ".wav", path_noisy + "noisy_" + str(i + 1) + ".wav", i + 1)

    # Adding the information to the excel file
    duration = round(librosa.get_duration(path=select_speech), 2) # Duration of speech
    passage_ID = speech_entries[ran_2].split('.')[0].split('_')[1] # Passage ID
    noise_channel = noise_entries[ran_4].split('.')[0]# Noise channel

    workfile1 = openpyxl.load_workbook(path3)

    sheet1 = workfile1.active

    sheet1.cell(row=i+2, column=1).value = i+1
    sheet1.cell(row=i+2, column=2).value = "speech_" + str(i + 1) + ".wav"
    sheet1.cell(row=i+2, column=3).value = duration
    sheet1.cell(row=i+2, column=4).value = "noisy | clean"
    sheet1.cell(row=i+2, column=5).value = "0 | -3 | -6 | -9 | 3 | 6 | 9"
    sheet1.cell(row=i+2, column=6).value = noise_file
    sheet1.cell(row=i+2, column=7).value = noise_channel
    sheet1.cell(row=i+2, column=8).value = speech_file
    sheet1.cell(row=i+2, column=9).value = passage_ID
    sheet1.cell(row=i+2, column=10).value = noise_start_time

    workfile1.save(path3)

    file_path = select_speech

    # Check if the file exists
    if os.path.exists(file_path):
        # Delete the file
        os.remove(file_path)

    # Update the user
    print("iteration: " + str(i + 1) + " noisy file: " + noise_file + " clean file: " + speech_file + " duration: " + str(duration) + " seconds", end = "\r")